In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

#### Task 1: Mark all launch sites on a map

In [3]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site','Lat','Long','class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index = False).first()
launch_sites_df = launch_sites_df[['Launch Site','Lat','Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [9]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Loop through each row in the DataFrame and add a marker with a visible label
for _, site in spacex_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 10pt; color: blue; text-align: center; padding: 10px">
                     {site['Launch Site']}
                     </div>""")
    ).add_to(site_map)

    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=1000,
        color='blue'
    ).add_to(site_map)

# Display the map
site_map

#### Task 2: Mark the success/failed launches for each site on the map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()

In [12]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def get_marker_color(cls):
    if cls == 1:
        return 'green'
    elif cls == 0:
        return 'red'
    else:
        return 'gray'

spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

In [14]:
# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

In [15]:
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

for _, site in spacex_df.iterrows():
    marker = folium.Marker(
        location = [site['Lat'],site['Long']],
        icon = folium.Icon(color = 'white', icon_color = site['marker_color'])
    )
# TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

#### TASK 3: Calculate the distances between a launch site to its proximities

In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [18]:
# Input launch site coordinates
launch_site_lat = float(input("Enter the latitude of the launch site: "))
launch_site_lon = float(input("Enter the longitude of the launch site: "))

# Input coastline coordinates
coastline_coords = []
num_coastlines = int(input("Enter the number of coastline points: "))

for i in range(num_coastlines):
    name = input(f"Enter the name of coastline {i+1}: ")
    lat = float(input(f"Enter the latitude of coastline {i+1}: "))
    lon = float(input(f"Enter the longitude of coastline {i+1}: "))
    coastline_coords.append({'name': name, 'Lat': lat, 'Lon': lon})

# Find the closest coastline
min_distance = float('inf')
closest_coastline = None

for coast in coastline_coords:
    distance = calculate_distance(launch_site_lat, launch_site_lon, coast['Lat'], coast['Lon'])
    if distance < min_distance:
        min_distance = distance
        closest_coastline = coast

# Print results
print(f"Closest coastline: {closest_coastline['name']}")
print(f"Distance: {min_distance:.2f} km")
print(f"Coordinates: Lat: {closest_coastline['Lat']}, Lon: {closest_coastline['Lon']}")

Enter the latitude of the launch site:  34.6329
Enter the longitude of the launch site:  -120.61066
Enter the number of coastline points:  2
Enter the name of coastline 1:  A
Enter the latitude of coastline 1:  34.64937
Enter the longitude of coastline 1:  -120.61907
Enter the name of coastline 2:  B
Enter the latitude of coastline 2:  34.63644
Enter the longitude of coastline 2:  -120.62482


Closest coastline: B
Distance: 1.35 km
Coordinates: Lat: 34.63644, Lon: -120.62482


In [19]:
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

distance_marker = folium.Marker(
    location=[closest_coastline['Lat'], closest_coastline['Lon']],
    icon=folium.DivIcon(
        icon_size=(150, 50),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{min_distance:.2f} KM</b></div>'
    )
).add_to(site_map)

# Add a marker for the launch site for reference
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Create a PolyLine object to connect the launch site and the closest coastline
coordinates = [
    [launch_site_lat, launch_site_lon],
    [closest_coastline['Lat'], closest_coastline['Lon']]
]

lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)

# Display the map
site_map

#### Drawing a line between Launch Site and Closest City, railways, Highway, etc...

In [20]:
# Input launch site coordinates
launch_site_lat = float(input("Enter the latitude of the launch site: "))
launch_site_lon = float(input("Enter the longitude of the launch site: "))

# Input points of interest
points_of_interest = []
num_points = int(input("Enter the number of points of interest: "))

for i in range(num_points):
    name = input(f"Enter the name of point of interest {i+1}: ")
    lat = float(input(f"Enter the latitude of point of interest {i+1}: "))
    lon = float(input(f"Enter the longitude of point of interest {i+1}: "))
    points_of_interest.append({'name': name, 'Lat': lat, 'Lon': lon})

# Find the closest point of interest
min_distance = float('inf')
closest_point = None

for point in points_of_interest:
    distance = calculate_distance(launch_site_lat, launch_site_lon, point['Lat'], point['Lon'])
    if distance < min_distance:
        min_distance = distance
        closest_point = point

# Initialize the map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Create and add a marker for the closest point of interest
distance_marker = folium.Marker(
    location=[closest_point['Lat'], closest_point['Lon']],
    icon=folium.DivIcon(
        icon_size=(150, 50),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{min_distance:.2f} KM</b></div>'
    )
).add_to(site_map)

# Add a marker for the launch site for reference
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Create a PolyLine object to connect the launch site and the closest point of interest
coordinates = [
    [launch_site_lat, launch_site_lon],
    [closest_point['Lat'], closest_point['Lon']]
]

lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(lines)

# Display the map
site_map

Enter the latitude of the launch site:  34.6329
Enter the longitude of the launch site:  -120.61066
Enter the number of points of interest:  1
Enter the name of point of interest 1:  34.63362
Enter the latitude of point of interest 1:  34.63362
Enter the longitude of point of interest 1:  -120.61904
